# Interacting with PyCodeGPT

In [1]:
import json
prompts = []
anss = []
with open('/home/scruple/NLP/base/datasets/CERT/NLTK/data/valid/try/output.jsonl', 'r') as valid:
    for val in valid.readlines():
        res = json.loads(val)
        prompt, ans = res['content'].split('\nANSWER:\n')
        prompt += '\nANSWER:\n'
        print(prompt+ans)
        prompts.append(prompt)
        anss.append(ans)
        # print(ans)



LIBRARY:
Flight

QUESTION:
ground transportation from airport in boston to downtown boston


ANSWER:
ColSet(AtomicColSet(COL_TRANSPORT()), AtomicRowPredSet(EQ_AIRPORT(CITY(boston))))


LIBRARY:
Flight

QUESTION:
information on flights from boston to washington


ANSWER:
AtomicRowPredSet(AtomicRowPred(EQ_DEPARTS(CITY(boston), ANY(), ANY(), ANY(), ANY()), EQ_ARRIVES(CITY(washington), ANY(), ANY(), ANY(), ANY())))


LIBRARY:
Flight

QUESTION:
kindly give me the type of aircraft used to fly from atlanta to denver


ANSWER:
PROJECT(AtomicColSet(COL_AIRCRAFT_TYPE()), AtomicRowPredSet(AtomicRowPred(EQ_DEPARTS(CITY(atlanta), ANY(), ANY(), ANY(), ANY()), EQ_ARRIVES(CITY(denver), ANY(), ANY(), ANY(), ANY()))))


LIBRARY:
Flight

QUESTION:
please inform me the type of aircraft used on a flight from atlanta to denver before 12 o clock noon


ANSWER:
PROJECT(AtomicColSet(COL_AIRCRAFT_TYPE()), AtomicRowPredSet(AtomicRowPred(EQ_DEPARTS(CITY(atlanta), Unit_Time_Set(MATCH_TIME_LT(TIME(12))), ANY(), AN

In [2]:
# %%writefile /content/PyCodeGPT/eval_human_eval.py
import os
import re
import argparse
import time
from tqdm import tqdm

from transformers import pipeline, set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.pipelines.base import Pipeline

# from human_eval.data import write_jsonl, read_problems

def load_generation_pipe(model_name_or_path: str, gpu_device: int=0):
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

    pipe = pipeline(
        'text-generation',
        model=model,
        tokenizer=tokenizer,
        device=gpu_device
    )

    print("load generation pipeline from {} over, vocab size = {}, eos id = {}, gpu device = {}.".format(
        model_name_or_path, len(tokenizer), tokenizer.eos_token_id, gpu_device)
    )

    return pipe

def extract_function_block(string):
    return re.split("\nclass|\ndef|\n#|\n@|\nprint|\nif", string)[0].rstrip()

def run_code_generation(pipe, prompt, num_completions=1, **gen_kwargs):
    set_seed(123)

    code_gens = pipe(prompt,
        num_return_sequences=num_completions,
        **gen_kwargs
    )

    return [extract_function_block(code_gen["generated_text"][len(prompt):]) for code_gen in code_gens]

class print_time:
    def __init__(self, desc):
        self.desc = desc

    def __enter__(self):
        print(self.desc)
        self.t = time.time()

    def __exit__(self, type, value, traceback):
        print(f'{self.desc} took {time.time()-self.t:.02f}s')

def evaluate_on_human_eval(
    model_name_or_path: str,
    temperature: float,
    top_p: float,
    num_samples_per_task: int,
    max_new_tokens: int,
    gpu_device: int,
    output_dir: str,
    ) -> str:

    pipe: Pipeline = load_generation_pipe(model_name_or_path, gpu_device=gpu_device)
    eval_name = f"human_eval.t{temperature}.p{top_p}.l{max_new_tokens}.n{num_samples_per_task}"

    if output_dir is None:
        if os.path.exists(model_name_or_path):
            output_dir = model_name_or_path
        else:
            raise ValueError("Output dir can't be null if you are not evaluation a local model.")

    os.makedirs(output_dir, exist_ok=True)
    saved_path = os.path.join(output_dir, f"{eval_name}.samples.jsonl")

    gen_kwargs = {
        "do_sample": True,
        "temperature": temperature,
        "max_new_tokens": max_new_tokens,
        "top_p": top_p,
        "top_k": 0,
        "pad_token_id": pipe.tokenizer.pad_token_id if pipe.tokenizer.pad_token_id else pipe.tokenizer.eos_token_id,
        "eos_token_id": pipe.tokenizer.eos_token_id
    }

    # problems = read_problems()
    samples = []
    generate_batch_size = min(50, num_samples_per_task)

    bos_token = pipe.tokenizer.bos_token if pipe.tokenizer.bos_token else pipe.tokenizer.eos_token

    # for task_id in tqdm(problems):
    #     # Strip operation is important as new tokenizer will not treat '\n' as a independent token
    #     prompt = problems[task_id]["prompt"].strip()

    #     for _ in range(num_samples_per_task // generate_batch_size):
    cnt = 0
    for prompt, ans in zip(prompts, anss):
        with print_time('sampling'):
            print('=' * 100)
            input_prompt = prompt
            print("input_prompt: \n" + input_prompt)
            gen_results = run_code_generation(pipe, input_prompt, num_completions=generate_batch_size, **gen_kwargs)
            print('=' * 100)
            print("generated code: \n")
            flag = False
            
            ans = ans.strip()
            for gen_result in gen_results:
                print('-' * 100)
                gen_result = gen_result.strip()
                if gen_result == "":
                    continue
                if gen_result == ans:
                    flag = True
                print("generated:", gen_result)
                print("answer:", ans)
            if flag:
                cnt+=1
        # break
    print(cnt, len(anss))
    print(cnt/len(anss))
          # samples.append(dict(task_id=task_id, completion=gen_result))
    
    # write_jsonl(saved_path, samples)
    # print("Run generation over, save {} samples to {}.".format(len(samples), saved_path))

evaluate_on_human_eval(
    model_name_or_path='/home/scruple/NLP/base/datasets/CERT/NLTK/experiments/normal_models/1x2x1x512-5e-4-0.1-1K_100_00K_2K-RS_1.0_0.5_1.0',
    temperature=0.8,
    top_p=0.95,
    num_samples_per_task=10,
    max_new_tokens=100,
    gpu_device=0,
    output_dir='results/',
)


load generation pipeline from /home/scruple/NLP/base/datasets/CERT/NLTK/experiments/normal_models/1x2x1x512-5e-4-0.1-1K_100_00K_2K-RS_1.0_0.5_1.0 over, vocab size = 32000, eos id = 0, gpu device = 0.
sampling
input_prompt: 

LIBRARY:
Flight

QUESTION:
ground transportation from airport in boston to downtown boston


ANSWER:

generated code: 

----------------------------------------------------------------------------------------------------
generated: ColSet(AtomicColSet(COL_TRANSPORT()), AtomicRowPredSet(EQ_AIRPORT(CITY(boston))))
answer: ColSet(AtomicColSet(COL_TRANSPORT()), AtomicRowPredSet(EQ_AIRPORT(CITY(boston))))
----------------------------------------------------------------------------------------------------
generated: ColSet(AtomicColSet(COL_TRANSPORT()), AtomicRowPredSet(EQ_AIRPORT(CITY(boston))))
answer: ColSet(AtomicColSet(COL_TRANSPORT()), AtomicRowPredSet(EQ_AIRPORT(CITY(boston))))
---------------------------------------------------------------------------------------

/home/scruple/miniconda3/envs/cert/lib/python3.7/site-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


generated code: 

----------------------------------------------------------------------------------------------------
generated: PROJECT(AtomicColSet(COL_AIRCRAFT_TYPE()), AtomicRowPredSet(AtomicRowPred(EQ_DEPARTS(CITY(atlanta), Unit_Time_Set(MATCH_TIME_LT(TIME(8am))), ANY(), ANY(), ANY()), EQ_ARRIVES(CITY(denver), ANY(), ANY(), ANY(), ANY()))))
answer: PROJECT(AtomicColSet(COL_AIRCRAFT_TYPE()), AtomicRowPredSet(AtomicRowPred(EQ_DEPARTS(CITY(atlanta), Unit_Time_Set(MATCH_TIME_LT(TIME(6pm))), ANY(), ANY(), ANY()), EQ_ARRIVES(CITY(denver), ANY(), ANY(), ANY(), ANY())), Eq_Airlines_IMP(AIRLINES(eastern))))
----------------------------------------------------------------------------------------------------
generated: PROJECT(AtomicColSet(COL_AIRCRAFT_TYPE()), AtomicRowPredSet(AtomicRowPred(EQ_DEPARTS(CITY(atlanta), Unit_Time_Set(MATCH_TIME_LT(TIME(8am))), ANY(), ANY(), ANY()), EQ_ARRIVES(CITY(denver), ANY(), ANY(), ANY(), ANY()))))
answer: PROJECT(AtomicColSet(COL_AIRCRAFT_TYPE()), Atomic

In [5]:
print('TextEdting:', 3/20)
print('ASTMatcher:', 13/93)
print('Flight:', 33/107)

TextEdting: 0.15
ASTMatcher: 0.13978494623655913
Flight: 0.308411214953271
